In [61]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [62]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [89]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import desc
from datetime import date
from datetime import datetime

In [90]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///hawaii.sqlite")

In [91]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)



In [92]:
# View all of the classes that automap found
Base.classes.keys()


['measurement', 'station']

In [93]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station


In [94]:
# Create our session (link) from Python to the DB
session = Session(engine)


In [95]:
testquery = session.query(measurement).first()
testquery.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x21ee0538520>,
 'prcp': 0.08,
 'date': '2010-01-01',
 'station': 'USC00519397',
 'id': 1,
 'tobs': 65.0}

In [96]:
testquery2 = session.query(station).first()
testquery2.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x21ee0493b80>,
 'longitude': -157.8168,
 'id': 1,
 'name': 'WAIKIKI 717.2, HI US',
 'latitude': 21.2716,
 'station': 'USC00519397',
 'elevation': 3.0}

# Exploratory Precipitation Analysis

In [100]:
# Find the most recent date in the data set.
recent_data = session.query(measurement).order_by(desc(measurement.date)).first()
recent_date = datetime.strptime(recent_data.__dict__['date'],'%yyyy-%mm-%dd')


ValueError: time data '2017-08-23' does not match format '%yyyy-%mm-%dd'

In [82]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database.
year_before = recent_date.year - 1
print('Last year:', year_before)
one_year_ago = date(year_before,today.month,today.day)
print('One year ago:', one_year_ago)
years_measurements = session.query(measurement).filter(measurement.date >= one_year_ago)

# Calculate the date one year from the last date in data set.


# Perform a query to retrieve the data and precipitation scores


# Save the query results as a Pandas DataFrame and set the index to the date column


# Sort the dataframe by date


# Use Pandas Plotting with Matplotlib to plot the data




AttributeError: 'str' object has no attribute 'year'

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()